In [1]:
import boto3
import pandas as pd
from urllib.request import urlopen
from PIL import Image

## 1. 이미지 url 불러오기

In [2]:
#첫 실행
'''
data_original = pd.read_csv("../../data/vlog_data_english.csv")
df = pd.DataFrame(data_original)
for index in range(len(df)):     #aws에서 정확히 찾기 위해, 이미지 화질 높이기
    df['thumbnail'][index] = df['thumbnail'][index].replace('default', 'mqdefault')
df['thumbnail_text']='not-yet'
df
'''

'\ndata_original = pd.read_csv("../../data/vlog_data_english.csv")\ndf = pd.DataFrame(data_original)\nfor index in range(len(df)):     #aws에서 정확히 찾기 위해, 이미지 화질 높이기\n    df[\'thumbnail\'][index] = df[\'thumbnail\'][index].replace(\'default\', \'mqdefault\')\ndf[\'thumbnail_text\']=\'not-yet\'\ndf\n'

In [8]:
data_original = pd.read_csv("../../data/english_data_including_thumbnailtext.csv")
df = pd.DataFrame(data_original)
df

,title,video_id,channel_name,channel_id,thumbnail,publish_time,view,thumbnail_text
0,VLOG: МЫ ТАК ПОДУМАЛИ! 25.05.21,kaibm6LSmb4,Katy LifeVlog,UCpm6vCgiehSKHtMFFS5ukiA,https://i.ytimg.com/vi/kaibm6LSmb4/mqdefault.jpg,2021-05-26T02:30:05Z,101580,"{'TextDetections': [{'DetectedText': 'Sweets',..."
1,RC Car with Train Horns on Golf Course,zYgAf2eu8sQ,Vlog Creations,UCR0VLWitB2xM4q7tjkoJUPw,https://i.ytimg.com/vi/zYgAf2eu8sQ/mqdefault.jpg,2021-05-24T16:00:21Z,1147867,{'TextDetections': [{'DetectedText': 'INSURANC...
2,Brought my FATHER&#39;s PILLOW to sleep | Ss V...,OtACAGFKpn8,Sambhavna Seth Entertainment,UChZ0JmGgyOWSU-1iXxPhsVA,https://i.ytimg.com/vi/OtACAGFKpn8/mqdefault.jpg,2021-05-26T06:25:15Z,429219,"{'TextDetections': [{'DetectedText': 'for', 'T..."
3,6AM PRODUCTIVE online school morning routine/v...,2_q4T1abM8Q,Kristy Bae,UCiLSb-cS_0pkjIYsm9oOB0w,https://i.ytimg.com/vi/2_q4T1abM8Q/mqdefault.jpg,2021-04-29T14:00:02Z,607709,"{'TextDetections': [{'DetectedText': 'I', 'Typ..."
4,My Funniest Vlog Ever 😂 Ayachi Thakur Vlogs,tHD5ajeSxKQ,Ayachi Thakur Vlogs,UC8RXP2epKG3uiPO5QYQyGqg,https://i.ytimg.com/vi/tHD5ajeSxKQ/mqdefault.jpg,2021-05-26T01:30:07Z,46413,"{'TextDetections': [], 'TextModelVersion': '3...."
...,...,...,...,...,...,...,...,...
8020,SI YO FUERA MADRE | Lyna Vlogs,HG6Qkqv5eBQ,Lyna Vlogs,UCRtkuS3Wz0hYgfJjpvEEP4Q,https://i.ytimg.com/vi/HG6Qkqv5eBQ/default.jpg,2017-10-08T22:39:42Z,4884575,not-yet
8021,| DHARAN | Dasain Vlog 2017 | Bhedetar,ZqPTPoyr2M4,Sisan Baniya,UC6SEUWgSEUOTKdwdRV7IRhA,https://i.ytimg.com/vi/ZqPTPoyr2M4/default.jpg,2017-10-09T06:36:39Z,296289,not-yet
8022,VLOG! ALA E TRUMP!,sA3Z_aHJrh8,MaxINFINITE,UCcLZdjaJEC_6RqfM3PhefIw,https://i.ytimg.com/vi/sA3Z_aHJrh8/default.jpg,2017-10-07T08:32:58Z,491894,not-yet
8023,VLOG #78: Hướng dẫn đi máy bay cho người đi lầ...,cwlD15TESUI,Yêu Máy Bay,UC6iYLWyzdYKMpnWbplhBrxg,https://i.ytimg.com/vi/cwlD15TESUI/default.jpg,2017-10-03T10:07:44Z,512585,not-yet


## 2. AWS API로 썸네일 속 텍스트 찾기
API 호출 회수를 줄이기 위해  
호출 결과를 우선 그대로 저장(이후 다른 코드에서 필요한 정보만 추출)

In [9]:
client=boto3.client('rekognition')

In [10]:
start_index = -1
for index in range(len(df)):
    if(df['thumbnail_text'][index]=='not-yet'):
        start_index = index
        break
print(start_index)

4618


In [ ]:
if(start_index==-1):
    print('already finished')
else:
    for index in range(start_index, len(df)):
        try:
            response = client.detect_text(Image={'Bytes': urlopen(df['thumbnail'][index]).read()})
            df['thumbnail_text'][index]=response
            print(index)
        except:
            df['thumbnail_text'][index]='error'
            print(index, 'error')
            pass

In [1]:
df[5000:5010]

NameError: name 'df' is not defined

## 3. CSV로 만들기

In [27]:
df.to_csv("../../data/english_data_including_thumbnailtext.csv", header=True, index=False)
df

,title,video_id,channel_name,channel_id,thumbnail,publish_time,view,thumbnail_text
0,VLOG: МЫ ТАК ПОДУМАЛИ! 25.05.21,kaibm6LSmb4,Katy LifeVlog,UCpm6vCgiehSKHtMFFS5ukiA,https://i.ytimg.com/vi/kaibm6LSmb4/mqdefault.jpg,2021-05-26T02:30:05Z,101580,"{'TextDetections': [{'DetectedText': 'Sweets',..."
1,RC Car with Train Horns on Golf Course,zYgAf2eu8sQ,Vlog Creations,UCR0VLWitB2xM4q7tjkoJUPw,https://i.ytimg.com/vi/zYgAf2eu8sQ/mqdefault.jpg,2021-05-24T16:00:21Z,1147867,{'TextDetections': [{'DetectedText': 'INSURANC...
2,Brought my FATHER&#39;s PILLOW to sleep | Ss V...,OtACAGFKpn8,Sambhavna Seth Entertainment,UChZ0JmGgyOWSU-1iXxPhsVA,https://i.ytimg.com/vi/OtACAGFKpn8/mqdefault.jpg,2021-05-26T06:25:15Z,429219,"{'TextDetections': [{'DetectedText': 'for', 'T..."
3,6AM PRODUCTIVE online school morning routine/v...,2_q4T1abM8Q,Kristy Bae,UCiLSb-cS_0pkjIYsm9oOB0w,https://i.ytimg.com/vi/2_q4T1abM8Q/mqdefault.jpg,2021-04-29T14:00:02Z,607709,"{'TextDetections': [{'DetectedText': 'I', 'Typ..."
4,My Funniest Vlog Ever 😂 Ayachi Thakur Vlogs,tHD5ajeSxKQ,Ayachi Thakur Vlogs,UC8RXP2epKG3uiPO5QYQyGqg,https://i.ytimg.com/vi/tHD5ajeSxKQ/mqdefault.jpg,2021-05-26T01:30:07Z,46413,"{'TextDetections': [], 'TextModelVersion': '3...."
...,...,...,...,...,...,...,...,...
4678,Science Fair,OdQxuxhn6_w,YouTube Movies,UCrkyaGALPDPqhzW4nC7E4mQ,https://i.ytimg.com/vi/OdQxuxhn6_w/mqdefault.jpg,2018-10-03T12:58:02Z,-2,{'TextDetections': [{'DetectedText': 'SCIENCE ...
4679,Scotty and the Secret History of Hollywood,TMsDIDEsNKw,YouTube Movies,UCwq_ssokmhMh2_zWFZjYRdQ,https://i.ytimg.com/vi/TMsDIDEsNKw/mqdefault.jpg,2018-11-13T02:00:58Z,-2,"{'TextDetections': [{'DetectedText': 'the', 'T..."
4680,DOROSŁOŚĆ 🎬 Piątek,1fIHs5oyMPU,PIĄTEK - serial oryginalny,UC23FduHce6_Sxlg-B6zLETw,https://i.ytimg.com/vi/1fIHs5oyMPU/mqdefault.jpg,2018-11-29T23:00:00Z,202990,"{'TextDetections': [{'DetectedText': 'PLATEK',..."
4681,PATRON ATEISTÓW 🎬 Piątek,FvwcVoeMZ14,PIĄTEK - serial oryginalny,UC23FduHce6_Sxlg-B6zLETw,https://i.ytimg.com/vi/FvwcVoeMZ14/mqdefault.jpg,2018-11-15T23:01:38Z,297391,"{'TextDetections': [{'DetectedText': 'PIATER',..."


## + 썸네일 url 부정확으로 검색 불가시, 해당 row 삭제

In [26]:
print(len(df))
for index in range(len(df['thumbnail'])):
    if(df['thumbnail_text'][index]=='error'):
        print(index, df['thumbnail'][index])
        df.drop(index, inplace=True)
print(len(df))

4683
1377 https://i.ytimg.com/vi/jsFjCAuOptE/mqdefault.jpg
1407 https://i.ytimg.com/vi/Onxg24j2W70/mqdefault.jpg
1409 https://i.ytimg.com/vi/ovBRnnf5sfE/mqdefault.jpg
1426 https://i.ytimg.com/vi/Vi0cSZNHR7E/mqdefault.jpg
1430 https://i.ytimg.com/vi/Ggr3a44VUWc/mqdefault.jpg
1438 https://i.ytimg.com/vi/3D6shBChUC0/mqdefault.jpg
1441 https://i.ytimg.com/vi/VipDZJ6gBZI/mqdefault.jpg
1465 https://i.ytimg.com/vi/lHp3zFdXNFY/mqdefault.jpg
1825 https://i.ytimg.com/vi/E3It61B8hTI/mqdefault.jpg
1889 https://i.ytimg.com/vi/kBOr73B6-H8/mqdefault.jpg
1891 https://i.ytimg.com/vi/sxf02lEhy9c/mqdefault.jpg
1916 https://i.ytimg.com/vi/dhq__Tk59gw/mqdefault.jpg
2249 https://i.ytimg.com/vi/4dTafIJ5ook/mqdefault.jpg
3219 https://i.ytimg.com/vi/rcccjM41r6A/mqdefault.jpg
4020 https://i.ytimg.com/vi/LFxrTvKHPGQ/mqdefault.jpg
4668


## 모델링 방법
2가지로 나누어 따로 진행  
feature 1 - 내용 : 제목이랑 똑같은 방식  
feature 2 - 면적 : 단어별 height x width 합  